In [1]:
#Importing Dependencies
import pandas as pd
import matplotlib.pyplot as plt

#Linear regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

#Data
from mega import model_df





In [4]:
model_df.tail()

,Gm#,W/L,D/N,H/A,Opp,COL_at_bats,COL_ba,COL_hits,COL_hr,COL_kk,COL_obp,COL_walks,Opp_at_bats,Opp_ba,Opp_hits,OPP_HR_Column,OPP_kk,Opp_obp,Opp_walks
1810,131,L,D,A,NYY,35,0.342857,12,5,6,0.410256,3,29,0.206897,6,0,8,0.323529,5
1811,132,W,N,H,MIA,34,0.264706,9,0,8,0.324324,3,28,0.214286,6,1,11,0.312500,4
1812,133,L,N,H,MIA,35,0.342857,12,2,5,0.428571,6,34,0.382353,13,4,8,0.405405,1
1813,134,8:40 pm,Game Preview and Matchups,H,MIA,33,0.242424,8,1,10,0.285714,2,30,0.300000,9,1,10,0.416667,6
1814,135,3:10 pm,Game Preview and Matchups,H,MIA,42,0.428571,18,2,7,0.489362,4,36,0.333333,12,3,10,0.390244,4


In [3]:
model_df["W/L"].unique()

array(['L', 'W', '8:40 pm', '3:10 pm'], dtype=object)

## Going Through Linear and Logistic Regression Models

### Linear

In [6]:
#The first step is getting the already preworked data into a useable format. 
#This includes the binary values and the categorical.

#Converting the binary columns with dummies.
dummies= pd.get_dummies(model_df[['W/L', 'D/N', 'H/A']], drop_first=True)
dummies

,W/L_8:40 pm,W/L_L,W/L_W,D/N_Game Preview and Matchups,D/N_N,H/A_H
0,False,True,False,False,False,False
1,False,False,True,False,True,False
2,False,False,True,False,True,False
3,False,False,True,False,False,True
4,False,False,True,False,True,True
...,...,...,...,...,...,...
1810,False,True,False,False,False,False
1811,False,False,True,False,True,True
1812,False,True,False,False,True,True
1813,True,False,False,True,False,True
